In [1]:
import numpy as np

import tflearn
from sklearn.utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

tflearn.activations
from sklearn.model_selection import train_test_split


In [12]:
def dense_to_one_hot(labels_dense, num_classes=10):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [11]:
def fit_and_predict(net, data, label):
    model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='./learn_logs/')
    model.fit({'input': data["X_train"]}, {'target': data["y_train"]},
              validation_set=({'input': data["X_test"]}, {'target': data["y_test"]}),
              n_epoch=100,
              snapshot_step=1000,
              show_metric=True,
              run_id=label,
              batch_size=256)

In [16]:
def task_1(activaction_='tanh'):
    network = input_data(shape=[None, 28, 28, 1], name='input')
    network = fully_connected(network, n_units=1024 * 4, activation='tanh')
    network = fully_connected(network, n_units=1024 * 2, activation=activaction_)
    network = fully_connected(network, n_units=1024 * 1, activation=activaction_)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='sgd', learning_rate=0.001, loss='categorical_crossentropy', name='target')
    return network

In [14]:

def task_2(activaction_='tanh'):
    network = input_data(shape=[None, 28, 28, 1], name='input')
    network = conv_2d(network, nb_filter=3, filter_size=3, strides=1, activation='relu')
    network = fully_connected(network, 1024 * 4, activation=activaction_)
    network = fully_connected(network, 1024 * 2, activation=activaction_)
    network = fully_connected(network, 1024 * 1, activation=activaction_)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='target')
    return network

In [15]:

def task_3():
    network = input_data(shape=[None, 28, 28, 1], name='input')

    network = conv_2d(network, nb_filter=3, filter_size=3, strides=1, activation='relu')
    network = max_pool_2d(network, kernel_size=3, strides=2)

    network = conv_2d(network, nb_filter=3, filter_size=3, strides=1, activation='relu')
    network = max_pool_2d(network, kernel_size=3, strides=2)

    network = fully_connected(network, 1024 * 4, activation='relu')
    network = fully_connected(network, 1024 * 2, activation='relu')
    network = fully_connected(network, 1024 * 1, activation='relu')

    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='target')
    return network

In [ ]:
def dop_task_1():
    network = input_data(shape=[None, 28, 28, 1], name='input')

    network = conv_2d(network, nb_filter=3, filter_size=3, strides=1, activation='relu')
    network = max_pool_2d(network, kernel_size=3, strides=2)

    network = conv_2d(network, nb_filter=3, filter_size=3, strides=1, activation='relu')
    network = max_pool_2d(network, kernel_size=3, strides=2)

    network = fully_connected(network, 1024 * 4, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024 * 2, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024 * 1, activation='relu')

    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.007, loss='categorical_crossentropy', name='target')
    return network

In [9]:
data = np.load("notMNIST.pickle", allow_pickle=True)

X, Y = data['test_dataset'], data['test_labels']
X_val, y_val = data['valid_dataset'][:1000], data['valid_labels'][:1000]
del data

In [10]:
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
# Split data on train and test
X, Y = shuffle(X, Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2, stratify=Y)
del X, Y

In [13]:
X_train = X_train.reshape([-1, 28, 28, 1])
y_train = dense_to_one_hot(y_train, 10)

X_test = X_test.reshape([-1, 28, 28, 1])
y_test = dense_to_one_hot(y_test)

X_val = X_val.reshape([-1, 28, 28, 1])
y_val = dense_to_one_hot(y_val)

data_dict = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "X_val": X_val,
    "y_val": y_val
}

In [ ]:
for activation in ['sigmoid', 'relu', 'tanh']:
    fit_and_predict(task_1(activation), data_dict, "_".join(["task_1", activation]))

In [ ]:
fit_and_predict(task_2(), data_dict, "_".join(["task_2", "tanh"]))

In [ ]:
fit_and_predict(task_3(), data_dict, "_".join(["task_3", ""]))

In [ ]:
fit_and_predict(dop_task_1(), data_dict, "_".join(["dop_task", ""]))

In [ ]:
fit_and_predict(task_3(), data_dict, "_".join(["task_3_withoutCUDA", ""])

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split



min_max_scaler = preprocessing.Normalizer()
df = pd.read_csv("spambase_new.csv", header=0)

Y = df["label"]
Y = np.array(Y)
del df['label']

X = df
X = np.array(X)
X = min_max_scaler.fit_transform(X)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

clf = LogisticRegression()
clf.fit(X_train, Y_train)
print 'score Scikit learn: ', clf.score(X_test, Y_test)

def Hypothesis(theta, x):
    def Sigmoid(z):
        G_of_Z = float(1.0 / float((1.0 + math.exp(-1.0 * z))))
        return G_of_Z

    z = 0
    for i in xrange(len(theta)):
        z += x[i] * theta[i]
    return Sigmoid(z)



def Gradient_Descent(X, Y, theta, m, alpha):
    def Cost_Function_Derivative(X, Y, theta, j, m, alpha):
        sumErrors = 0
        for i in xrange(m):
            xi = X[i]
            xij = xi[j]
            hi = Hypothesis(theta, X[i])
            error = (hi - Y[i]) * xij
            sumErrors += error
        m = len(Y)
        constant = float(alpha) / float(m)
        J = constant * sumErrors
        return J

    new_theta = []
    constant = alpha / m
    for j in xrange(len(theta)):
        CFDerivative = Cost_Function_Derivative(X, Y, theta, j, m, alpha)
        new_theta_value = theta[j] - CFDerivative
        new_theta.append(new_theta_value)
    return new_theta



def Logistic_Regression(X, Y, alpha, theta, num_iters):
    def Cost_Function(X, Y, theta, m):
        sumOfErrors = 0
        for i in xrange(m):
            xi = X[i]
            hi = Hypothesis(theta, xi)
            if Y[i] == 1:
                error = Y[i] * math.log(hi)
            elif Y[i] == 0:
                error = (1 - Y[i]) * math.log(1 - hi)
            sumOfErrors += error
        const = -1 / m
        J = const * sumOfErrors
        print 'cost is ', J
        return J

    def Declare_Winner(theta):
        score = 0
        winner = ""
        scikit_score = clf.score(X_test, Y_test)
        length = len(X_test)
        probas = []
        for i in xrange(length):
            proba = Hypothesis(X_test[i], theta)
            probas.append(proba)
            prediction = round(proba)
            answer = Y_test[i]
            if prediction == answer:
                score += 1

        my_score = float(score) / float(length)
        print 'Your score: ', my_score
        print 'Scikits score: ', scikit_score

        print "ROC_AUC_SCORE: ", roc_auc_score(y_true=Y_test, y_score=probas)

        fpr, tpr, _ = roc_curve(Y_test, probas)
        roc_auc = auc(fpr, tpr)

        import matplotlib.pylab as plt

        plt.figure()
        lw = 2
        plt.plot(fpr, tpr, color='darkorange',
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

    m = len(Y)
    for x in xrange(num_iters):
        new_theta = Gradient_Descent(X, Y, theta, m, alpha)
        theta = new_theta
        if x % 50 == 0:
            print x, Cost_Function(X, Y, theta, m)
    Declare_Winner(theta)


initial_theta = np.random.uniform(-0.01, 0.01, (len(df.columns),))
alpha = 0.8
iterations = 1000
Logistic_Regression(X, Y, alpha, initial_theta, iterations)
